# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Souza  ->  S. Souza  |  ['S. Souza']
G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
N. Deacon  ->  N. Deacon  |  ['N. Deacon']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
T. Herbst  ->  T. Herbst  |  ['T. Herbst']


N. Deacon  ->  N. Deacon  |  ['N. Deacon']
M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
Arxiv has 77 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.06357


extracting tarball to tmp_2507.06357... done.
Retrieving document from  https://arxiv.org/e-print/2507.06793


extracting tarball to tmp_2507.06793...

 done.


...

Found 180 bibliographic references in tmp_2507.06989/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.07093


extracting tarball to tmp_2507.07093...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.07093/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'sections/bhm_cartons/bhm_target_cartons_v0' from 'tmp_2507.07093/sections/bhm_cartons/bhm_target_cartons_v0.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'sections/mwm_cartons/mwm_cartons_appendix' from 'tmp_2507.07093/sections/mwm_cartons/mwm_cartons_appendix.tex'
  warnings.warn(LatexWarnin

Found 194 bibliographic references in tmp_2507.07093/main.bbl.
Issues with the citations
syntax error in line 473: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.06793-b31b1b.svg)](https://arxiv.org/abs/2507.06793) | **Spatially resolved H$α$ emission in B14-65666: compact starbursts, ionizing efficiency and gas kinematics in an advanced merger at the Epoch of Reionization**  |
|| C. Prieto-Jiménez, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-07-10*|
|*Comments*| *16 pages, 9 figures, accepted for publication in Astronomy and Astrophysics*|
|**Abstract**|            We present MIRI/JWST medium resolution spectroscopy (MRS) and imaging (MIRIM) of B14-65666, a Lyman-break and interacting galaxy at redshift $z$=7.15. We detect the H$\alpha$ line emission in this system, revealing a spatially-resolved structure of the H$\alpha$ emitting gas, which consists of two distinct galaxies, E and W, at a projected distance of 0.4". Galaxy E is very compact in the rest-frame UV, while W galaxy is more extended, showing a clumpy structure reminiscent of a tidal tail. The total H$\alpha$ luminosity implies that the system is forming stars at a Star Formation Rate (SFR) of 76$\pm$8 M$_{\odot}$ yr$^{-1}$ and 30$\pm$4 M$_{\odot}$ yr$^{-1}$ for E and W, respectively. The ionizing photon production efficiency is within the range measured in galaxies at similar redshifts. The high values derived for the H$\alpha$ equivalent widths (EW) and the distinct locations of the E and W galaxies in the $\log(\zeta_\mathrm{ion}$) $-$ EW (H$\alpha$) plane, indicate that the system is dominated by a young (less than 10 Myr) stellar population. The overall spectral energy distribution suggests that in addition to a young stellar population, the two galaxies may have mature stellar population and very different dust attenuation. The derived SFR and stellar masses identify the two galaxies as going through a starburst phase. The kinematics of the ionized gas traced by the H$\alpha$ line show a velocity difference of 175 $\pm$ 28 km s$^{-1}$ between the two components of B14-65666. The in-depth study of systems like B14-65666 reveal how galaxy mergers in the early Universe drive intense star formation, shape the interstellar medium, and influence the buildup of stellar mass, just 700 $-$ 800 Myr after the Big Bang.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.06863-b31b1b.svg)](https://arxiv.org/abs/2507.06863) | **The Spheroidal Bulge of the Milky Way: Chemodynamically Distinct from the Inner-Thick Disc and Bar**  |
|| S. Nepal, et al. -- incl., <mark>S. Souza</mark>, <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-07-10*|
|*Comments*| *Submitted. Abstract modified and abridged due to character limit*|
|**Abstract**|            Studying the composition and origin of the inner region of our Galaxy -- the "Galactic bulge" -- is crucial for understanding the formation and evolution of the Milky Way and other galaxies. We present new observational constraints based on a sample of around 18,000 stars in the inner Galaxy, combining Gaia DR3 RVS and APOGEE DR17 spectroscopy. Gaia-RVS complements APOGEE by improving sampling of the metallicity, [Fe/H] in the -2.0 to -0.5 dex range. This work marks the first application of Gaia-RVS spectroscopy to the bulge region, enabled by a novel machine learning approach (hybrid-CNN) that derives stellar parameters from intermediate-resolution spectra with precision comparable to APOGEE's infrared data. We performed full orbit integrations using a barred Galactic potential and applied orbital frequency analysis to disentangle the stellar populations in the inner Milky Way. For the first time, traced by the field stars, we are able to robustly identify the long-sought pressure supported bulge. We show this stellar population to be chemically and kinematically distinct from the other main components co-existing in the same region. The spheroidal bulge has a metallicity distribution function (MDF) peak at around -0.70 dex extending to solar value, is dominated by a high-[alpha/Fe] population with almost no dependency on metallicity, consistent with very rapid early formation, predating the thick disc and the bar. We find evidence that the bar has influenced the dynamics of the spheroidal bulge, introducing a mild triaxiality and radial extension. We identify a group of stars on X4 orbits, likely native to the early spheroid, as this population mimics the chemistry of the spheroidal bulge, with a minor contamination from the more metal-poor ([Fe/H] < -1.0) halo. We find the inner-thick disc to be kinematically hotter (mean Vphi ~125 km/s) than the local-thick disc. ...         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.06989-b31b1b.svg)](https://arxiv.org/abs/2507.06989) | **Sloan Digital Sky Survey-V: Pioneering Panoptic Spectroscopy**  |
|| J. A. Kollmeier, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>N. Deacon</mark>, <mark>M. Demianenko</mark>, <mark>K. El-Badry</mark>, <mark>M. Häberle</mark>, <mark>T. Herbst</mark> |
|*Appeared on*| *2025-07-10*|
|*Comments*| *76 pages, submitted to the Astronomical Journal*|
|**Abstract**|            The Sloan Digital Sky Survey-V (SDSS-V) is pioneering panoptic spectroscopy: it is the first all-sky, multi-epoch, optical-to-infrared spectroscopic survey. SDSS-V is mapping the sky with multi-object spectroscopy (MOS) at telescopes in both hemispheres (the 2.5-m Sloan Foundation Telescope at Apache Point Observatory and the 100-inch du Pont Telescope at Las Campanas Observatory), where 500 zonal robotic fiber positioners feed light from a wide-field focal plane to an optical (R$\sim 2000$, 500 fibers) and a near-infrared (R$\sim 22,000$, 300 fibers) spectrograph. In addition to these MOS capabilities, the survey is pioneering ultra wide-field ($\sim$ 4000~deg$^2$) integral field spectroscopy enabled by a new dedicated facility (LVM-I) at Las Campanas Observatory, where an integral field spectrograph (IFS) with 1801 lenslet-coupled fibers arranged in a 0.5 degree diameter hexagon feeds multiple R$\sim$4000 optical spectrographs that cover 3600-9800 angstroms. SDSS-V's hardware and multi-year survey strategy are designed to decode the chemo-dynamical history of the Milky Way Galaxy and tackle fundamental open issues in stellar physics in its Milky Way Mapper program, trace the growth physics of supermassive black holes in its Black Hole Mapper program, and understand the self-regulation mechanisms and the chemical enrichment of galactic ecosystems at the energy-injection scale in its Local Volume Mapper program. The survey is well-timed to multiply the scientific output from major all-sky space missions. The SDSS-V MOS programs began robotic operations in 2021; IFS observations began in 2023 with the completion of the LVM-I facility. SDSS-V builds upon decades of heritage of SDSS's pioneering advances in data analysis, collaboration spirit, infrastructure, and product deliverables in astronomy.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.07093-b31b1b.svg)](https://arxiv.org/abs/2507.07093) | **The Nineteenth Data Release of the Sloan Digital Sky Survey**  |
|| S. Collaboration, et al. -- incl., <mark>N. Deacon</mark>, <mark>M. Demianenko</mark>, <mark>K. El-Badry</mark>, <mark>M. Häberle</mark> |
|*Appeared on*| *2025-07-10*|
|*Comments*| *Submitted to AASJournals. 56 Pages, 9 Tables, 11 Figures*|
|**Abstract**|            Mapping the local and distant Universe is key to our understanding of it. For decades, the Sloan Digital Sky Survey (SDSS) has made a concerted effort to map millions of celestial objects to constrain the physical processes that govern our Universe. The most recent and fifth generation of SDSS (SDSS-V) is organized into three scientific ``mappers". Milky Way Mapper (MWM) that aims to chart the various components of the Milky Way and constrain its formation and assembly, Black Hole Mapper (BHM), which focuses on understanding supermassive black holes in distant galaxies across the Universe, and Local Volume Mapper (LVM), which uses integral field spectroscopy to map the ionized interstellar medium in the local group. This paper describes and outlines the scope and content for the nineteenth data release (DR19) of SDSS and the most substantial to date in SDSS-V. DR19 is the first to contain data from all three mappers. Additionally, we also describe nine value added catalogs (VACs) that enhance the science that can be conducted with the SDSS-V data. Finally, we discuss how to access SDSS DR19 and provide illustrative examples and tutorials.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.06357-b31b1b.svg)](https://arxiv.org/abs/2507.06357) | **Differentiable Stellar Atmospheres with Physics-Informed Neural Networks**  |
|| <mark>J. Li</mark>, M. Jian, Y.-S. Ting, G. M. Green |
|*Appeared on*| *2025-07-10*|
|*Comments*| *Accepted at the Workshop on Machine Learning for Astrophysics (ICML 2025)*|
|**Abstract**|            We present Kurucz-a1, a physics-informed neural network (PINN) that emulates 1D stellar atmosphere models under Local Thermodynamic Equilibrium (LTE), addressing a critical bottleneck in differentiable stellar spectroscopy. By incorporating hydrostatic equilibrium as a physical constraint during training, Kurucz-a1 creates a differentiable atmospheric structure solver that maintains physical consistency while achieving computational efficiency. Kurucz-a1 can achieve superior hydrostatic equilibrium and more consistent with the solar observed spectra compared to ATLAS-12 itself, demonstrating the advantages of modern optimization techniques. Combined with modern differentiable radiative transfer codes, this approach enables data-driven optimization of universal physical parameters across diverse stellar populations-a capability essential for next-generation stellar astrophysics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.06841-b31b1b.svg)](https://arxiv.org/abs/2507.06841) | **Measuring cosmic baryon density with FRB and GW data**  |
|| J.-G. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-10*|
|*Comments*| *13 pages, 3 figures*|
|**Abstract**|            The dispersion measure (DM) analysis of extragalactic fast radio bursts (FRBs) has accounted for all cosmic baryons but remains limited by systematic uncertainties from various parameter degeneracies. We show that the prominent degeneracy between the baryon density ($\Omega_{\rm b}$) and the Hubble constant ($H_0$) can be disentangled through an independent $H_0$ value uniquely inferred from the absolute luminosity distance measurements of gravitational-wave (GW) standard sirens. By combining $104$ localized FRBs with $47$ GW events, we obtain a robust, late-Universe measurement of \ob $=0.0488\pm0.0064$ ($1\sigma$), in concordance with early-Universe observations of CMB + BBN. Notably, incorporating GW data helps not only avoid biases induced by the $H_0$ tension, but also mitigate those from the parameters of diffuse baryon fraction and DM distribution models. Although the current precision ($\sim 13\%$) is limited by sample size, the growing detections of both FRBs and GWs will make their synergy a powerful probe of low-redshift cosmology.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.06793/./mirim_mrs_nircam2.png', 'tmp_2507.06793/./M_R_B14.png', 'tmp_2507.06793/./log_Ion-EW.png']
copying  tmp_2507.06793/./mirim_mrs_nircam2.png to _build/html/
copying  tmp_2507.06793/./M_R_B14.png to _build/html/
copying  tmp_2507.06793/./log_Ion-EW.png to _build/html/
exported in  _build/html/2507.06793.md
    + _build/html/tmp_2507.06793/./mirim_mrs_nircam2.png
    + _build/html/tmp_2507.06793/./M_R_B14.png
    + _build/html/tmp_2507.06793/./log_Ion-EW.png
found figures []
exported in  _build/html/2507.06863.md
found figures []
exported in  _build/html/2507.06989.md
found figures ['tmp_2507.07093/./figures/Flowchart.png', 'tmp_2507.07093/./figures/IPL3_cool_3visit_unbiased_best_RVs.png', 'tmp_2507.07093/./figures/helix_rgb.png']
copying  tmp_2507.07093/./figures/Flowchart.png to _build/html/
copying  tmp_2507.07093/./figures/IPL3_cool_3visit_unbiased_best_RVs.png to _build/html/
copying  tmp_2507.07093/./figures/helix_rgb.png to _build/html/
exported in  

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\CII}{[\ion{C}{II}]158\mum}$
$\newcommand{\oddpm}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \ -#2}}}$</div>



<div id="title">

# Spatially resolved H$\alpha$ emission in B14-65666: compact starbursts, ionizing efficiency and gas kinematics in an advanced merger at the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.06793-b31b1b.svg)](https://arxiv.org/abs/2507.06793)<mark>Appeared on: 2025-07-10</mark> -  _16 pages, 9 figures, accepted for publication in Astronomy and Astrophysics_

</div>
<div id="authors">

C. Prieto-Jiménez, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present MIRI/JWST medium resolution spectroscopy (MRS) and imaging (MIRIM) of B14-65666, a source identified as a Lyman-break and interacting galaxy at a redshift of $z$ = 7.15. We detect the H $\alpha$ line emission in this system, revealing a spatially-resolved structure of the H $\alpha$ emitting gas, which consists of two distinct galaxies, E and W, at a projected distance of 0.4 arcsec apart (i.e. 2.2 kpc).One of the galaxies (E) is very compact (upper limit for the effective radius of 63 pc) in the rest-frame ultraviolet light, while the other galaxy (W) is more extended (effective radius of 348 pc), showing a clumpy structure reminiscent of a tidal tail.The total H $\alpha$ luminosity implies that the system is forming stars at a rate of 76 ± 8 M $_{\odot}$ yr $^{-1}$ and 30 ± 4 M $_{\odot}$ yr $^{-1}$ for E and W galaxies, respectively. The ionizing photon production efficiency, $\log(\zeta_\mathrm{ion}$ ), for galaxies E and W has values of 25.1 ± 0.1 Hz erg $^{-1}$ and 25.5 ± 0.1 Hz erg $^{-1}$ , within the range measured in galaxies at similar redshifts. The high values derived for the H $\alpha$ equivalent widths (832 ± 100 and 536 ± 78 $Å$ ) and the distinct locations of the E and W galaxies in the $\log(\zeta_\mathrm{ion}$ ) $-$ equivalent width (H $\alpha$ ) plane, indicate that the system is dominated by a young (less than 10 Myr) stellar population.  The overall spectral energy distribution suggests that in addition to a young stellar population, the two galaxies may have mature (more than 100 Myr) stellar population and very different dust attenuation with galaxy E showing a larger attenuation (A $_{V}$ = 1.5 mag), compared to the almost dust-free (A $_{V}$ = 0.1 mag) galaxy W. The derived Star Formation Rate (SFR) and stellar masses identify the two galaxies as going through a starburst phase characterized by a specific SFR (sSFR) of 40 -- 50 Gyr $^{-1}$ . Galaxy E has an extreme stellar mass surface density (6 $\times$ 10 $^4$ M $_{\odot}$ pc $^{-2}$ ), close to that of the nuclei of low- $z$ galaxies while galaxy W (10 $^3$ M $_{\odot}$ pc $^{-2}$ ) is consistent with the surface densities measured in galaxies at these redshifts.The kinematics of the ionized gas traced by the H $\alpha$ line show a velocity difference of 175 $\pm$ 28 km s $^{-1}$ between the two components of B14-65666, and a broader profile for galaxy W (312 $\pm $ 44 km s $^{-1}$ ) relative to galaxy E (243 $\pm$ 41 km s $^{-1}$ ).The detailed study of B14-65666 shows the complex stellar and Interstellar Medium structure in merging galaxy systems was already in place by the Epoch of Reionization. The general properties of B14-65666 agree with those predicted for massive merging systems at redshifts of 7 and above in the FIRSTLIGHT cosmological simulations.The in-depth study of systems like B14-65666 reveal how galaxy mergers in the early Universe drive intense star formation, shape the interstellar medium, and influence the buildup of stellar mass, just 700 -- 800 Myr after the Big Bang.

</div>

<div id="div_fig1">

<img src="tmp_2507.06793/./mirim_mrs_nircam2.png" alt="Fig5" width="100%"/>

**Figure 5. -** _Top left_: MIRI Image F560W. The white circumference represents an aperture of radius 0.9 ". The black solid elliptical line represents the galaxies E+W aperture for the H$\alpha$ emission of the whole galaxy, the black dashed lines represent the apertures for galaxies E and W respectively. The white filled circle area on the bottom left represents the spatial resolution (PSF FWHM) of MIRI Image F560W. _Top right_: H$\alpha$ line map. The H$\alpha$ line map is generated by integrating H$\alpha$ line emission in the velocity range, -500 < _ v_[km s$^{-1}$] < 300. The white filled area represents the spatial resolution (PSF FWHM) of the MRS. _Bottom_: NIRCam cutouts in six filters, F115W to F444W (left to right and top to bottom). The images are homogenized to the PSF of MIRI Image, represented with the white filled circle on NIRCam F115W Image. (*fig:all*)

</div>
<div id="div_fig2">

<img src="tmp_2507.06793/./M_R_B14.png" alt="Fig3" width="100%"/>

**Figure 3. -** Mass-size relation for galaxies E (red star) and W (green star) in B14-65666,
    together with clumps of SMACS0723 at redshifts 6.4 to 8.5 (yellow squares, \citealt{Claeyssens_2023}) and a subsample of massive and compact galaxies from the CEERS program (purple circles, \citealt{baggen23}). The blue line represents the mass-size relation for galaxies (blue dots) at redshifts 4 to 10
(\citealt{langeroodi_2023})
    while the solid black line indicates the mass-size relation for redshifts 6 -- 9 derived for the NIRCam filter F150W (\citealt{allen2024galaxysizemassbuildup}). As B14-65666 is an interacting system, we also represent the mean values of the mass and size for local LIRGs and MS SFGs (\citealt{bellocchi_2022} and \citealt{leroy21}).
    Dotted gray lines represent constant stellar mass surface density for 10, 10$^2$, 10$^3$, 10$^4$ and 10$^5$ M$_\odot  \text{pc}^{-2}$. The red line indicates the observed maximum value of stellar mass surface density in clusters and nuclei of galaxies, 10$^{5.5}$ M$_\odot  \text{pc}^{-2}$. (*fig:mass-size*)

</div>
<div id="div_fig3">

<img src="tmp_2507.06793/./log_Ion-EW.png" alt="Fig1" width="100%"/>

**Figure 1. -** $\log(\zeta_\mathrm{ion})$ as a function of the H$\alpha$ equivalent width, both of them corrected from dust attenuation. The red star represents galaxy E of B14-65666 and the green star the galaxy W of B14-65666. Blue line represents the relation for galaxies at redshifts 3 to 7 (\citealt{Prieto-Lyon+23}). B14-65666 is compared with the galaxy MACS1149-JD1 (\citealt{alvarez-marquez+23c}), RXCJ0600-z6-3 (\citealt{gimenez-arteaga_2024}), mean of 102 Lyman-break galaxies at redshift 8 (\citealt{Stefanon+22}), MIDIS H$\alpha$ emitters at redshift 7-8 (\citealt{Rinaldi+23}), faint Ly$\alpha$ emitters (LAEs) at redshifts 4-5 (\citealt{Lam+19}), LAEs at 3.8-5 (\citealt{Maseda+20}), a LAE subsample at 4.9 (\citealt{harikane2018}), line emitters at 6.11 -- 6.35 (\citealt{sun_2023}) and LAEs at 6 (\citealt{Ning+23}). We also present the BPASS models of a binary stellar population for an instantaneous burst with ages from 1 to  20 Myr
   (crosses, from red to blue the ages are 20, 16, 13, 10, 8, 6, 4, 3, 2 and 1 Myr and connected with the black solid line)
  (\citealt{Stanway+Eldridge-23}). (*fig:log_Ion-EW*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.06793"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{Ç}[1]{\color{red}{{\bf CC:} #1} \color{black}}$
$\newcommand{\SN}[1]{\color{magenta}{{\bf SN:} #1} \color{black}}$
$\newcommand{\AP}[1]{\color{blue}{{\bf AP:} #1} \color{black}}$
$\newcommand{\fa}[1]{\color{teal}{{\bf AP:} #1} \color{black}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\teff}{\textit{T}_{\text{eff}}}$
$\newcommand{\teffsun}{\textit{T}_{\text{eff},\bigodot}}$
$\newcommand{\logg}{\text{log}(\textit{g})}$
$\newcommand{\mh}{[\text{M}/\text{H}]}$
$\newcommand{\feh}{[\text{Fe}/\text{H}]}$
$\newcommand{\alpham}{[\alpha/\text{M}]}$
$\newcommand{\alphafe}{[\alpha/\text{Fe}]}$
$\newcommand{\snr}{S/N}$
$\newcommand{\vrad}{V_{\text{rad}}}$
$\newcommand{\g}{G}$
$\newcommand{\bp}{\mathrm{\text{B}_\text{p}}}$
$\newcommand{\rp}{\mathrm{\text{R}_\text{p}}}$</div>



<div id="title">

# The Spheroidal Bulge of the Milky Way:\ Chemodynamically Distinct from the Inner-Thick Disc and Bar

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.06863-b31b1b.svg)](https://arxiv.org/abs/2507.06863)<mark>Appeared on: 2025-07-10</mark> -  _Submitted. Abstract modified and abridged due to character limit_

</div>
<div id="authors">

S. Nepal, et al. -- incl., <mark>S. Souza</mark>, <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** Studying the composition and origin of the inner region of our Galaxy -- the “Galactic bulge” -- is crucial for understanding the formation and evolution of the Milky Way and other galaxies. We present new observational constraints based on a sample of around 18,000 stars in the inner Galaxy, combining $_ Gaia_$ DR3 RVS and APOGEE DR17 spectroscopy. $_ Gaia_$ -RVS complements APOGEE by improving sampling of the metallicity, $\feh$ in the $-$ 2.0 to $-$ 0.5 dex range. This work marks the first application of $_ Gaia_$ -RVS spectroscopy to the bulge region, enabled by a novel machine learning approach ( ${\tt hybrid-CNN}$ ) that derives stellar parameters from intermediate-resolution spectra with precision comparable to APOGEE’s infrared data. We performed full orbit integrations using a barred Galactic potential and applied orbital frequency analysis to disentangle the stellar populations in the inner Milky Way. For the first time, traced by the field stars, we are able to robustly identify the long-sought pressure supported bulge. We show this stellar population to be chemically and kinematically distinct from the other main components co-existing in the same region. The spheroidal bulge has a metallicity distribution function (MDF) peak at around $-$ 0.70 dex extending to solar value, is dominated by a high- $\alphafe$ population with almost no dependency on metallicity, consistent with very rapid early formation, predating the thick disc and the bar. We find evidence that the bar has influenced the dynamics of the spheroidal bulge, introducing a mild triaxiality and radial extension. We identify a group of stars on $X_4$ orbits, likely native to the early spheroid, as this population mimics the chemistry of the spheroidal bulge, with a minor contamination from the more metal-poor ( $\feh$ < $-$ 1.0) halo. We find the inner-thick disc to be kinematically hotter ( $\overline{V_{\phi}}\approx$ 125 $\kms$ ) than the local-thick disc. The disc, chemically distinct from the spheroidal bulge and bar, is predominantly metal-poor with an MDF peak at $\feh\approx-0.45$ dex and includes a large fraction of stars with sub-solar $\feh$ and intermediate $\alphafe$ . In contrast to the spheroidal bulge, the disc $\alphafe$ shows a steeper decline with [ Fe/H ] , consistent with smaller star formation efficiency than that of the spheroidal bulge. Both the thick disc and the spheroidal bulge present vertical metallicity gradients. We find that the Galactic bar contains both metal-rich and metal-poor stars, as well as high- and low- $\alphafe$ in nearly equal measure, but their relative contributions vary significantly across different orbital families. The bar shows no strong metallicity trends in orbital extent or velocity dispersions and maintains a consistent elongated shape across all metallicities, indicating it is a well-mixed dynamical structure. Despite their spatial overlap, the spheroidal bulge, thick disc, and bar occupy distinct regions in both phase space and chemical abundance, indicating separate formation pathways. The stars with [ Fe/H ] $<-$ 1.0 and crossing the Galactic bulge are comprised by accreted populations primarily (70 \% ) belonging to the GES meger with a MDF peak at $-$ 1.30 dex and possibly a secondary merger remnant with MDF peak at $-$ 1.80 dex.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.06863"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\cise}{Computing in Science \& Engineering}$
$\newcommand{\ero}{\textit{eROSITA}\xspace}$
$\newcommand{\gai}{\textit{Gaia}\xspace}$</div>



<div id="title">

# Sloan Digital Sky Survey-V: Pioneering Panoptic Spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.06989-b31b1b.svg)](https://arxiv.org/abs/2507.06989)<mark>Appeared on: 2025-07-10</mark> -  _76 pages, submitted to the Astronomical Journal_

</div>
<div id="authors">

J. A. Kollmeier, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>N. Deacon</mark>, <mark>M. Demianenko</mark>, <mark>K. El-Badry</mark>, <mark>M. Häberle</mark>, <mark>T. Herbst</mark>

</div>
<div id="abstract">

**Abstract:** The _Sloan Digital Sky Survey V_ (SDSS-V) is pioneering panoptic spectroscopy: it is the first all-sky, multi-epoch, optical-to-infrared spectroscopic survey. SDSS-V is mapping the sky with multi-object spectroscopy (MOS) at telescopes in both hemispheres (the 2.5-m Sloan Foundation Telescope at Apache Point Observatory and the 100-inch du Pont Telescope at Las Campanas Observatory), where 500 zonal robotic fiber positioners feed light from a wide-field focal plane to an optical (R $\sim 2000$ , 500 fibers) and a near-infrared (R $\sim 22,000$ , 300 fibers) spectrograph. In addition to these MOS capabilities, the survey is pioneering ultra wide-field ( $\sim$ 4000 deg $^2$ ) integral field spectroscopy enabled by a new dedicated facility (LVM-I) at Las Campanas Observatory, where an integral field spectrograph (IFS) with 1801 lenslet-coupled fibers arranged in a 0.5 degree diameter hexagon feeds multiple R $\sim$ 4000 optical spectrographs that cover 3600-9800 Å.  SDSS-V's hardware and multi-year survey strategy are designed to decode the chemo-dynamical history of the Milky Way Galaxy and tackle fundamental open issues in stellar physics in its _Milky Way Mapper_ program, trace the growth physics of supermassive black holes in its _Black Hole Mapper_ program, and understand the self-regulation mechanisms and the chemical enrichment of galactic ecosystems at the energy-injection scale in its _Local Volume Mapper_ program.  The survey is well-timed to multiply the scientific output from major all-sky space missions. The SDSS-V MOS programs began robotic operations in 2021; IFS observations began in 2023 with the completion of the LVM-I facility.  SDSS-V builds upon decades of heritage of SDSS's pioneering advances in data analysis, collaboration spirit, infrastructure, and product deliverables in astronomy.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.06989"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\mathrm{eff}}}$
$\newcommand{\Teff}{T_{\mathrm{eff}}}$
$\newcommand{\logg}{log~g}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\oh}{[O/H]}$
$\newcommand{\mgh}{[Mg/H]}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\vsini}{V\sin{i}}$
$\newcommand{\vmicro}{\xi}$
$\newcommand{\afe}{[\alpha/Fe]}$
$\newcommand{\jmk}{(J-K_{s})}$
$\newcommand{\Ks}{K_{s}}$
$\newcommand{\fout}{f_\mathrm{out}}$
$\newcommand{\Gmk}{(G-K_s)}$
$\newcommand{\MK}{\mathrm{M}_{\mathrm{K_s}}}$
$\newcommand{\Mgrad}{\Delta[Fe/H]/\DeltaR}$
$\newcommand{\mygrad}{-0.066}$
$\newcommand{\Vgrad}{\Delta[Fe/H]/\DeltaZ}$
$\newcommand{\fehex}{\Delta[Fe/H]_{\text{excess}}}$
$\newcommand{\acronym}[1]{{\small{#1}}}$
$\newcommand{\Msun}{M\textsubscript{\(\odot\)}}$
$\newcommand{\project}[1]{\textsl{#1}}$
$\newcommand{\gaia}{\project{Gaia}}$
$\newcommand{\WISE}{\project{WISE}}$
$\newcommand{\tmass}{\project{2MASS}}$
$\newcommand{\thecannon}{\project{The~Cannon}}$
$\newcommand{\thepayne}{\project{The~Payne}}$
$\newcommand{\rave}{\project{\acronym{RAVE}}}$
$\newcommand{\galah}{\project{\acronym{GALAH}}}$
$\newcommand{\ges}{\project{GES}}$
$\newcommand{\apogee}{\project{\acronym{APOGEE}}}$
$\newcommand{\aspcap}{\project{\acronym{ASPCAP}}}$
$\newcommand{\astronn}{\texttt{astroNN}}$
$\newcommand{\lamost}{\project{\acronym{LAMOST}}}$
$\newcommand{\hipparcos}{\project{Hipparcos}}$
$\newcommand{\epic}{\project{K2/EPIC}}$
$\newcommand{\sdss}{\project{\acronym{SDSS}}}$
$\newcommand{\tgas}{\project{\acronym{TGAS}}}$
$\newcommand{\hmodel}{hierarchical model}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\hii}{\hbox{{\rm H}\kern 0.1em{\sc ii}{\rm }}}$
$\newcommand{\sean}[1]{{\color{green} \textbf{#1}}}$
$\newcommand{\chris}[1]{{\color{blue} \textbf{#1}}}$
$\newcommand{\carrie}[1]{{\color{orange}{\textbf{#1}}}}$
$\newcommand{\khcom}[1]{{\color{red}{\textbf{#1}}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand\code{#1}$</div>



<div id="title">

# The Nineteenth Data Release of the Sloan Digital Sky Survey 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.07093-b31b1b.svg)](https://arxiv.org/abs/2507.07093)<mark>Appeared on: 2025-07-10</mark> -  _Submitted to AASJournals. 56 Pages, 9 Tables, 11 Figures_

</div>
<div id="authors">

S. Collaboration, et al. -- incl., <mark>N. Deacon</mark>, <mark>M. Demianenko</mark>, <mark>K. El-Badry</mark>, <mark>M. Häberle</mark>

</div>
<div id="abstract">

**Abstract:** Mapping the local and distant Universe is key to our understanding of it. For decades, the Sloan Digital Sky Survey (SDSS) has made a concerted effort to map millions of celestial objects to constrain the physical processes that govern our Universe. The most recent and fifth generation of SDSS (SDSS-V) is organized into three scientific "mappers". Milky Way Mapper (MWM) that aims to chart the various components of the Milky Way and constrain its formation and assembly, Black Hole Mapper (BHM), which focuses on understanding supermassive black holes in distant galaxies across the Universe, and Local Volume Mapper (LVM), which uses integral field spectroscopy to map the ionized interstellar medium in the local group. This paper describes and outlines the scope and content for the nineteenth data release (DR19) of SDSS and the most substantial to date in SDSS-V. DR19 is the first to contain data from all three mappers. Additionally, we also describe nine value added catalogs (VACs) that enhance the science that can be conducted with the SDSS-V data. Finally, we discuss how to access SDSS DR19 and provide illustrative examples and tutorials.

</div>

<div id="div_fig1">

<img src="tmp_2507.07093/./figures/Flowchart.png" alt="Fig6" width="100%"/>

**Figure 6. -** This flowchart contains the decision tree used to determine which pipeline and reported parameters Astra are preferred for an individual unique source (i.e., with a unique SDSS\_ID). (*fig:flow_lite*)

</div>
<div id="div_fig2">

<img src="tmp_2507.07093/./figures/IPL3_cool_3visit_unbiased_best_RVs.png" alt="Fig2" width="100%"/>

**Figure 2. -** The relationship between the median visit SNR versus the unbiased radial velocity scatter for stars with the following cuts: $T_{\rm eff} < 6250$ K, $\logg \: > 1.5$, and unbiased RV scatter $< 1.0$ km/s. The underlying color map shows the number of stars in each logarithmic 2-dimension bin. Each line shows the median unbiased RV scatter in each SNR bin for SDSS Data Releases 12 (orange dot dashed line), 17 (red dashed line), and 19 (black line). The The data is binned by logarithmic x-coordinate (e.g. 10, 20, 30...100, 200 etc.) The horizontal dotted lines are at 100 m/s and 30 m/s respectively. (*fig:apogeerv*)

</div>
<div id="div_fig3">

<img src="tmp_2507.07093/./figures/helix_rgb.png" alt="Fig5" width="100%"/>

**Figure 5. -** Composite red (R), green (G), blue (B) map of the Helix nebula constructed by using spectral windows that capture the three emission lines given in the title. Fibers with known issues have been removed. We include fibers that catch contaminating stars. (*fig:helix*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.07093"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

140  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
